In [13]:
import numpy as np
from model_functions import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [2]:
#uncomment to run and create the example file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'example.pickle')

In [41]:
subject_data, freqs, targets = load_psds('example.pickle')

In [42]:
subject_data

[array([[[6.14861099e-10, 3.40339776e-10, 5.00924259e-10, ...,
          1.17614760e-12, 7.50624506e-13, 8.06907890e-13],
         [5.81309405e-10, 2.94068107e-10, 5.21400195e-10, ...,
          5.30984294e-13, 4.80516596e-13, 4.10975314e-13],
         [5.12000130e-10, 2.62413180e-10, 4.87293913e-10, ...,
          1.51338141e-13, 2.54862389e-13, 3.86776853e-13],
         ...,
         [4.80143226e-10, 2.74793885e-10, 5.34987300e-10, ...,
          1.69476877e-13, 1.73362092e-13, 1.46770581e-13],
         [4.43988094e-10, 2.73320014e-10, 5.13747692e-10, ...,
          1.49597610e-13, 1.17049866e-13, 1.19984950e-13],
         [5.06961913e-10, 2.35460886e-10, 4.87525211e-10, ...,
          9.98483727e-14, 9.19944615e-14, 1.02781144e-13]],
 
        [[1.04350237e-09, 5.19633410e-10, 2.26409724e-10, ...,
          8.48503665e-13, 7.53621046e-13, 8.69097012e-13],
         [1.10471083e-09, 5.01699811e-10, 2.67671625e-10, ...,
          4.23018161e-13, 4.89041125e-13, 3.58745947e-13],
       

In [43]:
freqs

array([ 0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,  2.5 ,
        2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,  4.5 ,  4.75,
        5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,  6.75,  7.  ,
        7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,  9.  ,  9.25,
        9.5 ,  9.75, 10.  , 10.25, 10.5 , 10.75, 11.  , 11.25, 11.5 ,
       11.75, 12.  , 12.25, 12.5 , 12.75, 13.  , 13.25, 13.5 , 13.75,
       14.  , 14.25, 14.5 , 14.75, 15.  , 15.25, 15.5 , 15.75, 16.  ,
       16.25, 16.5 , 16.75, 17.  , 17.25, 17.5 , 17.75, 18.  , 18.25,
       18.5 , 18.75, 19.  , 19.25, 19.5 , 19.75, 20.  , 20.25, 20.5 ,
       20.75, 21.  , 21.25, 21.5 , 21.75, 22.  , 22.25, 22.5 , 22.75,
       23.  , 23.25, 23.5 , 23.75, 24.  , 24.25, 24.5 , 24.75, 25.  ,
       25.25, 25.5 , 25.75, 26.  , 26.25, 26.5 , 26.75, 27.  , 27.25,
       27.5 , 27.75, 28.  , 28.25, 28.5 , 28.75, 29.  , 29.25, 29.5 ,
       29.75, 30.  , 30.25, 30.5 , 30.75, 31.  , 31.25, 31.5 , 31.75,
       32.  , 32.25,

In [44]:
targets

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [45]:
test_labels = align_test_labels(classes=['A','C'])

In [46]:
features, targets = remove_class(subject_data,targets,'F')

In [47]:
features, targets = remove_test(features,targets,test_labels)

In [48]:
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,8.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

In [49]:
def kNN_cross(rbps,targets,n_neighbors):
    confusion_matrices = []
    labels = np.unique(targets)
    for i in range(len(targets)):
        train_X, train_y = train_prep(rbps,targets,exclude=i,flatten_final=True)
        test_X = rbps[i].reshape(rbps[i].shape[0],-1)
        test_y = targets[i]*np.ones(rbps[i].shape[0])

        scaler = StandardScaler()
        train_X = scaler.fit_transform(train_X)
        
        ThreeNN = KNeighborsClassifier(n_neighbors=n_neighbors)
        ThreeNN.fit(train_X, train_y)
        
        test_X = scaler.transform(test_X)
        
        confusion_matrices += [confusion_matrix(test_y,ThreeNN.predict(test_X),labels=labels)]
    confusion_matrices = np.array(confusion_matrices)
    total_confusion = np.sum(confusion_matrices, axis= 0)
    return {'acc':accuracy(total_confusion), 'sens':sensitivity(total_confusion), 'spec':specificity(total_confusion), 'f1':f1(total_confusion)}

In [50]:
ThreeNN_metrics = kNN_cross(rbps,targets,n_neighbors=3)
ThreeNN_metrics

{'acc': 0.6565260058881256,
 'sens': 0.6559466019417476,
 'spec': 0.6572036905606813,
 'f1': 0.6731008717310087}